In [1]:
import pandas as pd
import os


def createEpisodes(csv_path):

    #Reads csv and store in data frame
    df = pd.read_csv(csv_path)
    dirName = 'Episodes'


    # Create target Directory
    try: 
        os.mkdir(dirName)
        print("Directory " , dirName ,  " Created ") 
    except FileExistsError:
        print("Directory " , dirName ,  " already exists")


    #Gets all unique ids from csv
    id = df['stop_id'].unique()

    #Create csv for each unique set of ids and their corresponding rows 
    for i in id:
        idframe =  df[df["id"] == i]
        idframe= idframe[["id","lat","lon","time"]]
        idframe = idframe.dropna()
        idframe.to_csv("./Episodes/ID_"+str(i)+"_Episode.csv", index=False)





        
    

    
    



    

createEpisodes('stops.txt')

Directory  Episodes  Created 
   stop_id      stop_name  stop_desc   stop_lat   stop_lon
0    18848       Clínicas        NaN        NaN -46.671108
1    18848  Vila Madalena        NaN -23.546498 -46.691141
next
   stop_id   stop_name  stop_desc   stop_lat   stop_lon
2    18850  Consolação        NaN -23.558094 -46.660205
next
   stop_id  stop_name  stop_desc   stop_lat   stop_lon
3    18851  Conceição        NaN -23.635039 -46.641239
next
   stop_id  stop_name  stop_desc   stop_lat   stop_lon
4    18852  Jabaquara        NaN -23.645996 -46.641027
next
   stop_id  stop_name  stop_desc   stop_lat   stop_lon
5    18853  São Judas        NaN -23.625882 -46.640936
next
   stop_id stop_name  stop_desc   stop_lat   stop_lon
6    18854     Saúde        NaN -23.618245 -46.639139
next
   stop_id        stop_name  stop_desc   stop_lat   stop_lon
7    18855  Praça Da Árvore        NaN -23.610583 -46.637918
next
   stop_id   stop_name  stop_desc   stop_lat   stop_lon
8    18856  Santa Cruz        